In [1]:
import os, glob
import subprocess
import tempfile
from tqdm import tqdm
import copy
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray
from pyproj import Proj, Transformer, Geod
from rioxarray import merge
from osgeo import gdal
from joblib import Parallel, delayed
from oggm import utils
import gzip
import json
import topojson
import shapely
from shapely.geometry import Polygon, MultiPolygon
from shapely.wkt import loads, dumps

In [16]:
# RGI
#version = '62'
version = '70G'
if version == '62':
    name_rgi_id = 'RGIId'
    name_column_area = 'Area'
    name_column_name = 'Name'
elif version == '70G':
    name_rgi_id = 'rgi_id'
    name_column_area = 'area_km2'
    name_column_name = 'glac_name'
else: raise ValueError('Version is wrong. Ciao')
OUT_PATH = f"/home/maffe/PycharmProjects/iceboost_webapp"

In [17]:
list_gdfs = []

for rgi in range(1,20):

    FILE_SHP_RGI = utils.get_rgi_region_file(region=f"{rgi:02d}", version=version)
    print(rgi, FILE_SHP_RGI)

    gdf = gpd.read_file(FILE_SHP_RGI)

    gdf = gdf[[name_rgi_id, name_column_name, name_column_area, 'geometry']]

    gdf.rename(columns={
        name_rgi_id: 'rgi_id',
        name_column_name: 'name',
        name_column_area: 'area'
    }, inplace=True)
    
    #if name_rgi_id == 'RGIId':
    #    gdf.rename(columns={'RGIId': 'rgi_id'}, inplace=True)

    gdf_rounded = gdf.copy()

    gdf_rounded['geometry'] = gdf_rounded['geometry'].apply(lambda geom: shapely.set_precision(geom, 0.00001))

    list_gdfs.append(gdf_rounded)

print('Combining geodataframes')
combined_gdf = pd.concat(list_gdfs, ignore_index=True)
print('Done.')

print('Creating geojson')
geojson_file = f"{OUT_PATH}/all_geometries_rgi{version}_cleaned.geojson"
print(OUT_PATH)
print(geojson_file)

combined_gdf.to_file(geojson_file, driver='GeoJSON')

tippecanoe_path_out = f"/home/maffe/PycharmProjects/iceboost_webapp/tiles/RGI{version}/geometries/"
tippecanoe_file_out = f"geoms_rgi{version}.mbtiles"
tippecanoe_out_layer_name = f"geoms_rgi{version}"
tippecanoe_out = tippecanoe_path_out + tippecanoe_file_out

# Construct the Tippecanoe command
tippecanoe_command = f"tippecanoe -o {tippecanoe_out} -l {tippecanoe_out_layer_name} -zg {geojson_file} --force --preserve-input-order"

# Run Tippecanoe command via subprocess
subprocess.run(tippecanoe_command, shell=True, check=True)

# remove geojson_file
os.remove(geojson_file)
print("Deleted: ", geojson_file)

1 /home/maffe/OGGM/rgi/RGIV70G/RGI2000-v7.0-G-01_alaska/RGI2000-v7.0-G-01_alaska.shp
2 /home/maffe/OGGM/rgi/RGIV70G/RGI2000-v7.0-G-02_western_canada_usa/RGI2000-v7.0-G-02_western_canada_usa.shp
3 /home/maffe/OGGM/rgi/RGIV70G/RGI2000-v7.0-G-03_arctic_canada_north/RGI2000-v7.0-G-03_arctic_canada_north.shp
4 /home/maffe/OGGM/rgi/RGIV70G/RGI2000-v7.0-G-04_arctic_canada_south/RGI2000-v7.0-G-04_arctic_canada_south.shp
5 /home/maffe/OGGM/rgi/RGIV70G/RGI2000-v7.0-G-05_greenland_periphery/RGI2000-v7.0-G-05_greenland_periphery.shp
6 /home/maffe/OGGM/rgi/RGIV70G/RGI2000-v7.0-G-06_iceland/RGI2000-v7.0-G-06_iceland.shp
7 /home/maffe/OGGM/rgi/RGIV70G/RGI2000-v7.0-G-07_svalbard_jan_mayen/RGI2000-v7.0-G-07_svalbard_jan_mayen.shp
8 /home/maffe/OGGM/rgi/RGIV70G/RGI2000-v7.0-G-08_scandinavia/RGI2000-v7.0-G-08_scandinavia.shp
9 /home/maffe/OGGM/rgi/RGIV70G/RGI2000-v7.0-G-09_russian_arctic/RGI2000-v7.0-G-09_russian_arctic.shp
10 /home/maffe/OGGM/rgi/RGIV70G/RGI2000-v7.0-G-10_north_asia/RGI2000-v7.0-G-10_no

/home/maffe/PycharmProjects/iceboost_webapp/tiles/RGI70G/geometries/all_geometries_rgi70G_cleaned.geojson:6: ignoring dimensions beyond two
In JSON object [-155.90416,67.41608,0.0]
In JSON object {"type":"Feature","properties":{"rgi_id":"RGI2000-v7.0-G-01-00001","name":null,"area":0.23919916103073499},"geometry":{"type":"Polygon","coordinates":[[[-155.90416,67.41608,0.0],[-155.90345,67.41584,0.0],[-155.90295,67.41566,0.0],[-155.90235,67.41546,0.0],[-155.90175,67.41528,0.0],[-155.9015,67.41517,0.0],[-155.90115,67.41496,0.0],[-155.90111,67.41491,0.0],[-155.90097,67.4147,0.0],[-155.90096,67.41446,0.0],[-155.901,67.41426,0.0],[-155.90127,67.41409,0.0],[-155.90163,67.41396,0.0],[-155.90208...
274531 features, 203410191 bytes of geometry, 3694498 bytes of separate metadata, 12492526 bytes of string pool
Choosing a maxzoom of -z6 for features about 7975 feet (2431 meters) apart
Choosing a maxzoom of -z10 for resolution of about 323 feet (98 meters) within features
tile 4/11/6 size is 681287 w

Deleted:  /home/maffe/PycharmProjects/iceboost_webapp/tiles/RGI70G/geometries/all_geometries_rgi70G_cleaned.geojson


In [15]:
# Antarctic Peninsula

In [15]:
from shapely.validation import make_valid
from shapely.validation import explain_validity

#PENINSULA_PATH = f"/media/maffe/nvme/Antarctic_peninsula_geometries/final_product/"
PENINSULA_PATH = f"/media/maffe/nvme/antarctic_peninsula_from_bedmachinev4/"

peninsula = gpd.read_file(PENINSULA_PATH+"antarctic_peninsula.shp")

if not peninsula.crs == "EPSG:4326":
    peninsula = peninsula.to_crs("EPSG:4326")

peninsula = peninsula[['geometry']]

# correct some geometries
peninsula['geometry'] = peninsula['geometry'].apply(
    lambda geom: make_valid(geom) if not geom.is_valid else geom
)

peninsula_rounded = peninsula.copy()

#n_invalid_before = (~peninsula_rounded.is_valid).sum()
#invalid_geoms = peninsula_rounded.loc[~peninsula_rounded.is_valid, 'geometry']
#reasons = invalid_geoms.apply(explain_validity)
#for idx, reason in reasons.items():
#    print(f"Index {idx}: {reason}")
#print(n_invalid_before)
#input('wait')

peninsula_rounded['geometry'] = peninsula_rounded['geometry'].apply(lambda geom: shapely.set_precision(geom, 0.00001))

print('Creating geojson')
geojson_file = f"{PENINSULA_PATH}peninsula_cleaned.geojson"
print(PENINSULA_PATH)
print(geojson_file)

peninsula_rounded.to_file(geojson_file, driver='GeoJSON')

tippecanoe_path_out = f"{PENINSULA_PATH}"
tippecanoe_file_out = f"peninsula.mbtiles"
tippecanoe_out_layer_name = f"peninsula"
tippecanoe_out = tippecanoe_path_out + tippecanoe_file_out

# Construct the Tippecanoe command
tippecanoe_command = f"tippecanoe -o {tippecanoe_out} -l {tippecanoe_out_layer_name} -zg {geojson_file} --force"

# Run Tippecanoe command via subprocess
subprocess.run(tippecanoe_command, shell=True, check=True)

# remove geojson_file
os.remove(geojson_file)
print("Deleted: ", geojson_file)

Creating geojson
/media/maffe/nvme/antarctic_peninsula_from_bedmachinev4/
/media/maffe/nvme/antarctic_peninsula_from_bedmachinev4/peninsula_cleaned.geojson
Deleted:  /media/maffe/nvme/antarctic_peninsula_from_bedmachinev4/peninsula_cleaned.geojson


50 features, 546985 bytes of geometry, 66 bytes of separate metadata, 0 bytes of string pool
Choosing a maxzoom of -z0 for features about 1086978 feet (331311 meters) apart
Choosing a maxzoom of -z5 for resolution of about 8538 feet (2602 meters) within features
  95.8%  5/10/24  
